<a href="https://colab.research.google.com/github/Likithareddy99/Missing_lab_tests/blob/main/Missing_Lab_Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers datasets
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForSequenceClassification


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import io
from google.colab import files
import json
import torch

In [3]:
#code to generate structured data using named-entity recognition 

def MissingTests(df1, df_icd, predicted_label):

  #initializing bert model 
  
  tokenizer = AutoTokenizer.from_pretrained("samrawal/bert-base-uncased_clinical-ner")
  model = AutoModelForTokenClassification.from_pretrained("samrawal/bert-base-uncased_clinical-ner")
  Bert_ner = pipeline('ner',model=model,tokenizer=tokenizer,grouped_entities=True,ignore_subwords=True)

  result = [] #nested list having dictionary 

  #Working on sample data of first 1000 rows only
  for i in df1['LONG_TITLE'][:1000]:
    result.append(Bert_ner(i))

  entities=[]
  for i in result:
    for j in i:
      entities.append([j['entity_group'],j['word']])

  Entity_table = pd.DataFrame(entities,columns=['entity_type','entity'])   
  Entity_table = Entity_table.drop_duplicates()

  Entity_table.reset_index(drop=True, inplace=True)
  Entity_table.reset_index()



  #Processing complete data requires more time and processing power. So, we're using already processed data 
  result_test={}
  for i in range(len(predicted_label)):
    for j in range(len(df_icd)):
      if df_icd['ICD_code'][j].__contains__(predicted_label[i]):
        result_test[df_icd['Entity'][j]]=df_icd["test"][j]
  print("\n complete tests required to identify the disease", result_test)
  
  #to store using json file format 
  filename = 'result.json'          #use the file extension .json
  with open(filename, 'w') as file_object:  #open the file in write mode
    json.dump(result_test, file_object) 

  return result_test





def AutomaticDiag(input):

  #Model Initialization
  tokenizer = AutoTokenizer.from_pretrained("bvanaken/CORe-clinical-diagnosis-prediction")
  model = AutoModelForSequenceClassification.from_pretrained("bvanaken/CORe-clinical-diagnosis-prediction")

  tokenized_input = tokenizer(input, return_tensors="pt")
  output = model(**tokenized_input)

  predictions = torch.sigmoid(output.logits)
  predicted_labels = [model.config.id2label[_id] for _id in (predictions > 0.3).nonzero()[:, 1].tolist()] 
  

  #code to check what tests were performed on the patient 
  
  tokenizer_ner = AutoTokenizer.from_pretrained("samrawal/bert-base-uncased_clinical-ner")
  model_ner = AutoModelForTokenClassification.from_pretrained("samrawal/bert-base-uncased_clinical-ner")
  Bert_ner = pipeline('ner',model=model_ner,tokenizer=tokenizer_ner,grouped_entities=True,ignore_subwords=True)   
  result = Bert_ner(input)
  entities=[]
  for j in result:
    entities.append([j['entity_group'],j['word']])

  Entity_table = pd.DataFrame(entities,columns=['entity_type','entity'])  

  Entity_table = Entity_table.drop_duplicates()
  Entity_table.reset_index(drop=True, inplace=True)
  Entity_table.reset_index()

  tests_performed = []
  for i in range(len(Entity_table['entity_type'])):
    if Entity_table['entity_type'][i] == 'test':
      tests_performed.append(Entity_table['entity'][i])

  tests_performed #storing in global variable to find missing lab tests by comparision 
  return predicted_labels, tests_performed




#function for coparing the results 

def comparision(test_performed, test_required):

  list3 = []
  suggested_tests = []

  for key in test_required:
    str1 = test_required[key].replace('[','')
    str2 = str1.replace(']','')

    list1 = str2.split("'")
    new_list = list(filter(lambda item: len(item) >= 1, list1))
    list3+=new_list
    
    list2 = list3
    for ele in list3:
      if  ele == ' ' or "\\" in r"%r" % ele:
        list2.remove(ele)

  tests_required2 = list2
  for elem in test_performed:
    if elem in list2:
      tests_required2.remove(elem)

  return tests_required2



def main(user_input):
  Tests_performed1=[]
  predicted_label1=[]
  test_required1 = {}

  
  df1 = pd.read_csv('https://raw.githubusercontent.com/Likithareddy99/Missing_lab_tests/main/D_ICD_DIAGNOSES.csv')
  

  df2 = pd.read_csv('https://raw.githubusercontent.com/Likithareddy99/Missing_lab_tests/main/Disease_icd.csv')


  inp = user_input

  #predicted label which we got from the automatic diagnsis 
  # predicted_label1=['427','482','790']
  predicted_label1, Tests_performed1 = AutomaticDiag(inp)   
  #passing the predicted label as argument to the Missing lab test 
  test_required1 = MissingTests(df1,df2, predicted_label1)
  print("\n the tests performed on patients are;",Tests_performed1)

  print("\n\n The missing lab tests to be performed are: \n",comparision(Tests_performed1, test_required1))
    
#Function Call 






In [4]:
ngrokToken = '2FV2D230ltMQHRjIdqBMJyB5JX4_2GXkSrxcqpmgdrYPTPeWg'
ngrokTokenCmd = 'ngrok authtoken '+ngrokToken

!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!mkdir templates
!mkdir static

!wget -q https://raw.githubusercontent.com/Likithareddy99/Missing_lab_tests/main/form.html
!wget -q https://raw.githubusercontent.com/Likithareddy99/Missing_lab_tests/main/result.html
!wget -q https://raw.githubusercontent.com/Likithareddy99/Missing_lab_tests/main/app.css
!wget -q https://raw.githubusercontent.com/Likithareddy99/Missing_lab_tests/main/search.png
!wget -q https://raw.githubusercontent.com/Likithareddy99/Missing_lab_tests/main/bg_1.png

!mv *.html templates/
!mv *.css static/
!mv *.png static/

mkdir: cannot create directory ‘templates’: File exists
mkdir: cannot create directory ‘static’: File exists


In [9]:
from pyngrok import ngrok
get_ipython().system_raw(ngrokTokenCmd)

public_url = ngrok.connect(5000)
public_url

<NgrokTunnel: "http://184c-34-68-191-214.ngrok.io" -> "http://localhost:5000">

In [10]:
from sys import argv
from flask import Flask
from flask import Flask, render_template, redirect, url_for, request
app = Flask(__name__)

"""@app.route("/")
def hello():
    retString = 'Hello visitor'
    return retString"""

@app.route('/')
def form():
    return render_template('form.html')
 
@app.route('/data/', methods = ['POST', 'GET'])
def data():
    if request.method == 'GET':
        return f"The URL /data is accessed directly. Try going to '/form' to submit form"
    if request.method == 'POST':
        form_data = request.form
        final_data = main(form_data['Input'])
        print(final_data)
        return render_template('result.html',result = form_data)

if __name__ == "__main__":
    app.run(host="0.0.0.0")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [30/Sep/2022 18:59:28] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Sep/2022 18:59:29] "GET /static/bootstrap.min.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [30/Sep/2022 18:59:29] "GET /static/app.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Sep/2022 18:59:29] "GET /static/search.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Sep/2022 18:59:29] "GET /static/bg_1.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Sep/2022 18:59:29] "GET /favicon.ico HTTP/1.1" 404 -
